# Análisis de regresión logística

In [ ]:
# @title Instalar y cargar paquetes
install.packages(c("blorr"))
library(tidyverse)
library(blorr)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘numDeriv’, ‘SparseM’, ‘MatrixModels’, ‘minqa’, ‘nloptr’, ‘RcppEigen’, ‘carData’, ‘abind’, ‘pbkrtest’, ‘quantreg’, ‘lme4’, ‘car’, ‘gridExtra’, ‘lest’, ‘Rcpp’


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


# Ejercicio

Un grupo de investigadores e investigadoras esperan que el estrés la ansiedad y la sintomatología de depresión contribuyan positivamente en los problemas de sueño de trabajadores médicos hospitalarios.

HT: El estrés, la ansiedad y depresión contribuyen a los problemas de sueño de las/los trabajadores médicos de hospital


*   $y$: Problemas de sueño (PS) [Cualitativa ordinal; 0 = No, 1 = Sí]
*   $x_1$: Estrés (E)
*   $x_2$: Ansiedad (A)
*   $x_3$: Depresión (D)

$Estrés \\
H_{0}: β_{E} = 0 \\
H_{1}: β_{E} > 0 \\
Ansiedad \\
H_{0}: β_{A} = 0 \\
H_{1}: β_{A} > 0 \\
Depresión \\
H_{0}: β_{D} = 0 \\
H_{1}: β_{D} > 0$

Ecuación del modelo de regresión logística:

$Logit (PS) = β_{0} - β_{E}X_{E} + β_{A}X_{A} + β_{D}X_{D}$

In [ ]:
# @title Cargar datos
df = haven::read_sav("https://github.com/renatoparedes/EstadisticaYPsicologiaMatematica/raw/main/INEE/Clase15_BaseRegresionLogistica.sav")
print(df, n = 10)

# A tibble: 100 × 7
   Código Sexo        Edad ProbSueño Estrés Ansiedad Depresión
   <chr>  <dbl+lbl>  <dbl> <dbl+lbl>  <dbl>    <dbl>     <dbl>
 1 001    0 [Mujer]     40 1 [Sí]         5        7         3
 2 002    1 [Hombre]    26 1 [Sí]         8        6         1
 3 003    1 [Hombre]    19 0 [No]         2        3         1
 4 004    1 [Hombre]    35 0 [No]         2        1         2
 5 005    1 [Hombre]    35 0 [No]         8        2         5
 6 006    0 [Mujer]     40 1 [Sí]        10        4         7
 7 007    0 [Mujer]     58 0 [No]         8        4         0
 8 008    0 [Mujer]     33 0 [No]         2        9         2
 9 009    1 [Hombre]    32 1 [Sí]         8        9         3
10 010    0 [Mujer]     53 1 [Sí]         8       11         6
# ℹ 90 more rows


In [ ]:
# @title Estimar el modelo
df %>%
  glm(ProbSueño ~ Estrés + Ansiedad + Depresión,
      family = "binomial",
      data = .) -> model1

In [ ]:
# @title $Prueba\ de\ Ommnibus$
blorr::blr_model_fit_stats(model1) %>% unclass() %>% as_tibble() -> ajuste
ajuste %>% select(starts_with("lr_"))

lr_ratio,lr_pval,lr_df
<dbl>,<dbl>,<dbl>
28.565,2.763898e-06,3


$H_{0}: \beta_{E} = \beta_{A} = \beta_{D} = 0 \\
H_{1}: Al\ menos\ una\ \beta\ es\ diferente\ de\ 0$

A partir de la prueba Omnibus, se observa que al menos una beta es diferente de cero, por lo que el modelo es adecuado para ser interpretado, $\chi^2_{(3)} = 28.565, p < .001$

In [ ]:
# @title $R^2\ de\ Nagelkerke$
ajuste %>% select(nagelkerke)

nagelkerke
<dbl>
0.3320105




*   Magnitud: grande
*   % de Verosimilitud: 33.2

El pseudo $R^2$ de Nagelkerke es grande y explica el 33.2% de la verosimilitud de tener un problema de sueño,$R^2_{Nagelkerke} = .332$.



In [ ]:
# @title $Prueba\ de\ Hosmer\ y\ Lemeshow$
blorr::blr_test_hosmer_lemeshow(model1) -> hl
hl[-1] %>% as_tibble()

chisq_stat,df,pvalue
<dbl>,<dbl>,<dbl>
1.720681,8,0.9884107


$H_{0}: valores\ observados = valores\ pronosticados \\
H_{1}: valores\ observados \neq valores\ pronosticados$

A partir de la prueba de Hosmer y Lemeshow, se observa que los valores observados y estimados son iguales, $\chi^2_{(8)} = 1.721, p = .988$

In [ ]:
# @title $Tabla\ de\ clasificación (aka\ Confusion\ matrix)$
blorr::blr_confusion_matrix(model1, cutoff = 0.5) -> mc
mc$conf_matrix %>% as_tibble()

Prediction,Reference,n
<chr>,<chr>,<int>
0,0,42
1,0,12
0,1,16
1,1,30



*   La cantidad de aciertos cuando $y = 0 (No)$ es 42.
*   La cantidad de fracasos cuando $y = 0 (No)$ es 12.
*   La cantidad de fracasos cuando $y = 1 (Sí)$ es 16.
*   La cantidad de aciertos cuando $y = 1 (Sí)$ es 30.




In [ ]:
# @title $Especificidad, Sensitividad\ y\ Accuracy\ (Porcentaje\ Global)$
mc[-3] %>% as_tibble() %>% select(accuracy, sensitivity, specificity)

accuracy,sensitivity,specificity
<dbl>,<dbl>,<dbl>
0.72,0.6521739,0.7777778


A nivel global, el modelo es adecuado para estimar los problemas de sueño ($Accuracy\ o\ Porcentaje\ global = 72\% > 65\%$). No obstante, el modelo es mejor estimando la ausencia ($Especificidad = 77.8\% > 65\%$) que la presencia ($Sensitividad = 65.2\% > .65\%$) de un problema de sueño.

In [ ]:
# @title $Coeficientes$
model1 %>% broom::tidy() %>% mutate(p.unilateral = p.value/2)

term,estimate,std.error,statistic,p.value,p.unilateral
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),-2.88375669,0.70102692,-4.1136176,3.895064e-05,1.947532e-05
Estrés,0.18067643,0.11390465,1.5862076,1.126922e-01,5.634611e-02
Ansiedad,0.22914056,0.09764158,2.3467519,1.893786e-02,9.468931e-03
Depresión,0.08139973,0.09883809,0.8235664,4.101860e-01,2.050930e-01


Ten en cuena que, dado que son hipótesis unilaterales, se ha creado una columna mas con los valores p unilaterales ($p.unilateral$ en la tabla de arriba). Con esta información, podemos pasar a analizar el resultado.

|-|Estrés|Ansiedad|Depresión|
|-|-|-|-|
|Sig|.056 > .05|.01 < .05|.205 > .05|
|Sentido|Positivo|Positivo|Positivo|

Ecuación del modelo

$logit(PS) = -2.88 + .18(E) + .23(A) + .08(D)$

A partir del análisis de regresión logística, se observa que existe un modelo adecuado con un tamaño del efecto grande que explica el 33.2% de la verosimilitud de tener un problema de sueño, $R^2_{Nagelkerke} = .332, \chi^2_{(3)} = 28.565, p < .001$. Específicamente, se encontró que, si bien la ansiedad, $B = .23, EE_{B} = .10, W_{(1)} = 2.35, p = .01$, resultó ser un predictor estadísticamente significativo y positivo de la ocurrencia de un problema de sueño, el estrés, $B = .18, EE_{B} = .11, W_{(1)} = 1.59, p = .056$, y la depresión, $B = .08, EE_{B} = .10, W_{(1)} = .82, p = .205$, no resultaron predictores estadísticamente significativos.

In [ ]:
# @title $Odd\ Ratio$
model1 %>% broom::tidy(exponentiate = T, conf.int = T)

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.05592428,0.70102692,-4.1136176,3.895064e-05
Estrés,1.19802747,0.11390465,1.5862076,1.126922e-01
Ansiedad,1.25751879,0.09764158,2.3467519,1.893786e-02
Depresión,1.08480444,0.09883809,0.8235664,4.101860e-01


Al colocar en la función $exponentiate = T, conf.int = T$ modificamos la columna de $estimate$ de modo que ahora nos indica el valor de $Odd\ Ratio$. Esto se debe a que los odds simplemente son el resultado de la función exponencial de los betas.

El coeficiente mas importante es el de la ansiedad, dado que tiene el odd ratio de mayor nivel, $Odd\ ratio = 1.26$. Así, por cada punto adicional en ansiedad, hay 1.26 veces mas posibilidades de desarrollar un problema de sueño que a no desarrollarlo.

De igual forma, se puede aplicar la siguiente fórmula: $(Odd\ ratio - 1) * 100$

Con el propósito de obtener el porcentaje de contribución de cada variable independiente.

In [ ]:
model1 %>% broom::tidy(exponentiate = T, conf.int = T) %>% mutate(perc = (estimate - 1) * 100)

term,estimate,std.error,statistic,p.value,conf.low,conf.high,perc
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.05592428,0.70102692,-4.1136176,3.895064e-05,0.01261783,0.2017084,-94.407572
Estrés,1.19802747,0.11390465,1.5862076,1.126922e-01,0.96019363,1.5066779,19.802747
Ansiedad,1.25751879,0.09764158,2.3467519,1.893786e-02,1.04974118,1.5450318,25.751879
Depresión,1.08480444,0.09883809,0.8235664,4.101860e-01,0.89309702,1.3213390,8.480444


El resultado de la fórmula anterior se encuentra en la última columna denominada $perc$.

Este análisis muestra que la ansiedad contribuye en un 25.8% a la ocurrencia de un problema de sueño.